# Chapter 02: 고객 지원 에이전트

이 노트북에서는 LangGraph를 사용하여 간단한 이커머스 고객 지원 에이전트를 구축합니다.

## 주요 내용
- LangGraph StateGraph 사용법
- 도구 정의 및 바인딩
- 에이전트 워크플로우 구성

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TeeDDub/AI-Agent-Engineering/blob/main/notebook/ch02_customer_support_agent.ipynb)


## 1. 패키지 설치


In [ ]:
!pip install -q langchain langchain-openai langgraph openai python-dotenv


## 2. API 키 설정

Google Colab의 Secrets 기능을 사용하거나 직접 입력하세요.


In [ ]:
import os

# 방법 1: Colab Secrets 사용 (권장)
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Colab Secrets에서 API 키를 불러왔습니다.")
except:
    pass

# 방법 2: 직접 입력 (Secrets 사용 불가 시)
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"  # 여기에 API 키 입력
    print("⚠️ API 키를 직접 입력했습니다. 보안에 주의하세요.")


## 3. 고객 지원 에이전트 구현


In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain.tools import tool
from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from langgraph.graph import StateGraph

# State 타입 정의
class AgentState(TypedDict):
    order: dict
    messages: Annotated[Sequence[BaseMessage], operator.add]

# 주문 취소 도구 정의
@tool
def cancel_order(order_id: str) -> str:
    """배송되지 않은 주문을 취소합니다."""
    # (여기서 실제 백엔드 API를 호출합니다)
    return f"주문 {order_id}이(가) 취소되었습니다."


In [ ]:
# 에이전트 구조 정의: LLM 호출, 도구 실행, 다시 LLM 호출
def call_model(state):
    msgs = state["messages"]
    order = state.get("order", {"order_id": "UNKNOWN"})

    # LLM 초기화 (gpt-5-mini 사용)
    llm = init_chat_model(model="gpt-5-mini", temperature=0)
    llm_with_tools = llm.bind_tools([cancel_order])  # 도구 바인딩

    # 시스템 프롬프트
    prompt = f'''당신은 이커머스 지원 에이전트입니다.
        주문 ID: {order['order_id']}
        고객이 취소를 요청하면 cancel_order(order_id)를 호출하고
        간단한 확인 메시지를 보내세요.
        그렇지 않으면 일반적으로 응답하세요.'''
    
    full = [SystemMessage(content=prompt)] + msgs

    # 1차 LLM 패스: 도구 호출 여부 결정
    first = llm_with_tools.invoke(full)
    out = [first]

    if getattr(first, "tool_calls", None):
        # cancel_order 도구 실행
        tc = first.tool_calls[0]
        result = cancel_order.invoke(tc["args"])
        out.append(ToolMessage(content=result, tool_call_id=tc["id"]))

        # 2차 LLM 패스: 최종 확인 텍스트 생성
        second = llm.invoke(full + out)
        out.append(second)

    return {"messages": out}


In [ ]:
# StateGraph로 에이전트 구조 연결
def construct_graph():
    g = StateGraph(AgentState)
    g.add_node("assistant", call_model)
    g.set_entry_point("assistant")
    return g.compile()

graph = construct_graph()
print("✅ 에이전트 그래프가 구성되었습니다.")


## 4. 에이전트 실행


In [ ]:
# 테스트 실행: 주문 취소 요청
example_order = {"order_id": "B73973"}
convo = [HumanMessage(content="주문 #B73973를 취소해주세요.")]

result = graph.invoke({"order": example_order, "messages": convo})

print("=" * 50)
print("🤖 고객 지원 에이전트 응답")
print("=" * 50)
for msg in result["messages"]:
    print(f"\n{msg.type}: {msg.content}")


In [ ]:
# 일반 질문 테스트
general_question = [HumanMessage(content="배송은 얼마나 걸리나요?")]
result2 = graph.invoke({"order": example_order, "messages": general_question})

print("=" * 50)
print("🤖 일반 질문 응답")
print("=" * 50)
for msg in result2["messages"]:
    print(f"{msg.type}: {msg.content}")
